# 0 설정, 설치 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/015GithubRepos/Dacon_sentence_classification')

In [3]:
pip install transformers

     |████████████████████████████████| 3.5 MB 7.9 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 6.8 MB 68.1 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# KoELECTRA

# from transformers import AutoModel, AutoTokenizer

# model = AutoModel.from_pretrained("klue/roberta-large")
# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

In [ ]:
# # RoBERTa large

# from transformers import AutoModel, AutoTokenizer

# model = AutoModel.from_pretrained("klue/roberta-large")
# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

In [ ]:
# # !pip install mxnet
# # !pip install gluonnlp pandas tqdm
# # !pip install sentencepiece
# # !pip install transformers
# # !pip install torch

# !pip install boto3==1.15.18
# !pip install gluonnlp==0.10.0
# !pip install mxnet==1.7.0.post2
# !pip install onnxruntime==0.5.0
# !pip install sentencepiece==0.1.96
# !pip install torch==1.10.1
# !pip install transformers==3.0.2

# # !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [4]:
import pandas as pd 
import numpy as np 
import re
from tqdm import tqdm
import time

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW, AutoModel, AutoModelForSequenceClassification
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split

import warnings 
warnings.filterwarnings("ignore")

## 경로지정

In [5]:
# local = 'C:/Users/posick/Desktop/Dacon/open/'
# local2 = 'C:/Users/201/Desktop/Dacon/'
suv = 'data/'
# colab = '/content/drive/MyDrive/Dacon/'

# 1 하이퍼 파라미터 설정

### epoch

In [6]:
# epoch 5만 해도될듯
num_epochs = 10

### batch_size

In [7]:
batch_size = 32

### learning rate

In [8]:
lr = 0.00001

# 2 Data 처리

## 텍스트 전처리

In [14]:
#%% data load 

def load_data(path):
    train = pd.read_csv(path+'train_data.csv')
    label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}
    train['label'] = train['label'].map(label_dict)
    test = pd.read_csv(path+'test_data.csv')
    sample_submission = pd.read_csv(path+'sample_submission.csv')
    
    return train,test,sample_submission 

def text_clean(df):
    df["premise_"] = "[CLS]" + df["premise"] + "[SEP]"
    df["hypothesis_"] = df["hypothesis"] + "[SEP]"
    df["text_sum"] = df.premise_ + " " + df.hypothesis_
    df = df[['text_sum','label']]
    return df 

train,test,sample_submission = load_data(suv)
clean_train,clean_test  = text_clean(train),text_clean(test)


## MyDataset 클래스 선언

In [11]:
#%% data loader 

class CustomDataset(Dataset):
  
  def __init__(self, dataset, option):
    self.dataset = dataset 
    self.option = option
    self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values  # numpy array
    text = row[0]
    # y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=70,
        pad_to_max_length=True,
        add_special_tokens=False
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    
    if self.option =='train':
        y = row[1]
        return input_ids, attention_mask, y

    return input_ids, attention_mask


## CrossValidation 인덱스 생성

In [12]:
#%% Cross validation 

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

folds=[]
for trn_idx, val_idx in skf.split(clean_train['text_sum'],clean_train['label']):
    folds.append((trn_idx,val_idx))


# 학습

## device 설정

In [13]:
device = torch.device("cuda")
device

device(type='cuda')

## 학습

In [55]:
for i,fold in enumerate(range(3, 5)):
    print(f'{i}, {fold}')

0, 3
1, 4


In [60]:
start_time = time.time()

13.271684408187866

In [ ]:
#%%
start_time = time.time()
#model=nn.DataParallel(model).to(device)
#optimizer = AdamW(model.parameters(), lr=0.001)
best_models = []

for i,fold in enumerate(range(3, 6)):
    print('==============================',fold+1,'fold start==============================')

    # CV용 model 새로 선언
    model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=3)
    model = nn.DataParallel(model).to(device)
    
    optimizer = AdamW(model.parameters(), lr=lr)
    
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]

    train_data = clean_train.loc[train_idx]
    val_data = clean_train.loc[valid_idx]

    train_dataset = CustomDataset(train_data, 'train')
    valid_dataset = CustomDataset(val_data, 'train')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    
    warmup_ratio = 0.1
    total_steps = len(train_loader) * num_epochs
    warmup_step = int(total_steps * warmup_ratio)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=total_steps)
    valid_loss_min = 0.4
    valid_acc_max = 0.8
    
    for epoch in range(num_epochs):  # epoch = 10
        print(f'epoch : {epoch}-----------------------------------------------------------------')
        batches = 0
        total_loss = 0.0
        correct = 0
        total =0
        model.train()  # train 모드로 변경
        
        # 학습데이터로 학습 #############################################################
        print(f'train 학습..........')
        for input_ids_batch, attention_masks_batch, y_batch in train_loader:
            optimizer.zero_grad()  # 그래디언트 초기화
            y_batch = y_batch.to(device)  # y_batch를 gpu 올림
            y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]  # 순전파
            loss = F.cross_entropy(y_pred, y_batch)  # loss 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
            total_loss += loss.item()
            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)
            batches += 1
            if batches % 100 == 0:
                acc = correct.float() / total
                print(f'Batch : {batches}, Train Loss: {total_loss:.4f}, Train Accuracy : {acc.item():.4f}')
        acc = correct.float() / total
        print(f'Batch 누적 : {batches}, Train Loss: {total_loss:.4f}, Train Accuracy : {acc.item():.4f}')
        # 학습데이터로 학습 #############################################################


        # 검증데이터로 검증 #############################################################
        val_loss = []
        val_acc = []
        print(f'validation 검증..........')
        for input_ids_batch, attention_masks_batch, y_batch in valid_loader:
            
            model.eval()
            with torch.no_grad():
                y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
                valid_loss = F.cross_entropy(y_pred,y_batch.to(device)).cpu().detach().numpy()

                preds = torch.argmax(y_pred,1)
                preds = preds.cpu().detach().numpy()
                y_batch = y_batch.cpu().detach().numpy()
                batch_acc = (preds==y_batch).mean()
                val_loss.append(valid_loss)
                val_acc.append(batch_acc)

        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)
        scheduler.step()
        print(f'Valid Loss: {val_loss:.6f}, Valid Accuracy : {val_acc:.6f}')
        print(f'Learning rate : {optimizer.param_groups[0]["lr"]:.6f}')
        # 검증데이터로 검증 #############################################################

        if valid_acc_max < val_acc:  # 이전보다 좋으면 best_models에 모델을 추가
            valid_acc_max = val_acc
            best_models.append(model)
            print(f'model save. the number of best_models: {len(best_models)}, model val acc : {val_acc:.6f}******************')
        print(f'--------------------------------------------------------------------------------')

print(f'elapsed time : {time.time() - start_time}')
# epoch 5만 해도될듯

============================== 4 fold start==============================


Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'cla

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

epoch : 0-----------------------------------------------------------------
train 학습..........
Batch : 100, Train Loss: 111.0504, Train Accuracy : 0.3425
Batch : 200, Train Loss: 221.8002, Train Accuracy : 0.3427
Batch : 300, Train Loss: 333.0980, Train Accuracy : 0.3399
Batch : 400, Train Loss: 444.4545, Train Accuracy : 0.3374
Batch : 500, Train Loss: 555.8282, Train Accuracy : 0.3354
Batch : 600, Train Loss: 666.9090, Train Accuracy : 0.3362
Batch : 625, Train Loss: 694.7627, Train Accuracy : 0.3369
validation 검증..........
valid Loss: 1.103537, valid_acc : 0.341333
learning rate : 1e-05
0 에폭 끝-----------------------------------------------------------------
epoch : 1-----------------------------------------------------------------
train 학습..........
Batch : 100, Train Loss: 93.1045, Train Accuracy : 0.5062
Batch : 200, Train Loss: 144.3757, Train Accuracy : 0.6591
Batch : 300, Train Loss: 185.6288, Train Accuracy : 0.7236
Batch : 400, Train Loss: 222.3179, Train Accuracy : 0.7602
Bat

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'cla

epoch : 0-----------------------------------------------------------------
train 학습..........
Batch : 100, Train Loss: 114.7085, Train Accuracy : 0.3259
Batch : 200, Train Loss: 228.8533, Train Accuracy : 0.3244
Batch : 300, Train Loss: 343.6157, Train Accuracy : 0.3243
Batch : 400, Train Loss: 458.2683, Train Accuracy : 0.3212
Batch : 500, Train Loss: 573.0619, Train Accuracy : 0.3232
Batch : 600, Train Loss: 687.1383, Train Accuracy : 0.3242
Batch : 625, Train Loss: 715.7962, Train Accuracy : 0.3239
validation 검증..........
valid Loss: 1.145578, valid_acc : 0.315486
learning rate : 1e-05
0 에폭 끝-----------------------------------------------------------------
epoch : 1-----------------------------------------------------------------
train 학습..........


In [17]:
test_dataset = CustomDataset(clean_test,'test')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

preds = []
for idx, m in enumerate(best_models): 
    print(f'{idx+1}/{len(best_models)}번째 모델 예측 진행중')
    bestm = m
    bestm.eval()
    answer = []
    with torch.no_grad():
        for input_ids_batch, attention_masks_batch in tqdm(test_loader):
            y_pred = bestm(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0].detach().cpu().numpy()
            answer.extend(y_pred.argmax(axis=1))
            
    preds.append(answer)

1/11번째 모델 예측 진행중


100%|██████████| 27/27 [00:12<00:00,  2.16it/s]


2/11번째 모델 예측 진행중


100%|██████████| 27/27 [00:12<00:00,  2.17it/s]


3/11번째 모델 예측 진행중


...

100%|██████████| 27/27 [00:12<00:00,  2.17it/s]


10/11번째 모델 예측 진행중


100%|██████████| 27/27 [00:12<00:00,  2.16it/s]


11/11번째 모델 예측 진행중


100%|██████████| 27/27 [00:12<00:00,  2.16it/s]


In [22]:
from collections import Counter

np_pred = np.array(preds).T

pred = []
for i in range(1666):
    cnt = Counter(np_pred[i])
    pred.append(cnt.most_common()[0][0])
    

In [26]:
label_dict1 = {0:"entailment" , 1: "contradiction" , 2:"neutral"}

sample_submission['label'] = [label_dict1[_] for _ in pred]

In [35]:
sample_submission

,index,label
0,0,contradiction
1,1,neutral
2,2,entailment
3,3,contradiction
4,4,contradiction
...,...,...
1661,1661,neutral
1662,1662,entailment
1663,1663,neutral
1664,1664,neutral


In [36]:
sample_submission.to_csv(suv + 'aaaaaaaaa.csv', index=False)

In [30]:
suv + 'kc_roberta-large_3fold.csv'

'data/kc_roberta-large_3fold.csv'

In [37]:
os.getcwd()

'/content/drive/MyDrive/015GithubRepos/Dacon_sentence_classification'

In [47]:
for _ in np_pred[:50]:
    print(f'{_}\t{Counter(_)}')

[1 1 1 1 1 1 1 1 1 1 1]	Counter({1: 11})
[2 2 2 2 2 2 2 2 2 2 2]	Counter({2: 11})
[0 0 0 0 0 0 0 0 0 0 0]	Counter({0: 11})
[1 1 1 1 1 1 1 1 1 1 1]	Counter({1: 11})
[1 1 1 1 1 1 1 1 1 1 1]	Counter({1: 11})
[2 2 2 2 2 2 2 2 2 2 2]	Counter({2: 11})
[1 1 1 2 2 2 2 2 2 2 2]	Counter({2: 8, 1: 3})
[1 1 1 1 1 1 1 0 0 0 0]	Counter({1: 7, 0: 4})
[0 0 0 0 0 0 0 0 0 0 0]	Counter({0: 11})
[1 1 1 2 2 2 2 1 1 1 1]	Counter({1: 7, 2: 4})
[1 1 1 1 1 1 1 1 1 1 1]	Counter({1: 11})
[0 0 0 0 0 0 0 0 0 0 0]	Counter({0: 11})
[1 1 1 1 1 1 1 1 1 1 1]	Counter({1: 11})
[0 0 0 0 0 0 0 0 0 0 0]	Counter({0: 11})
[2 2 2 2 2 2 2 2 2 2 2]	Counter({2: 11})
[2 2 2 2 2 2 2 2 2 2 2]	Counter({2: 11})
[2 2 2 0 0 0 0 0 0 0 0]	Counter({0: 8, 2: 3})
[2 2 2 2 2 2 2 2 2 2 2]	Counter({2: 11})
[1 1 1 1 1 1 1 1 1 1 1]	Counter({1: 11})
[2 2 2 2 2 2 2 2 2 2 2]	Counter({2: 11})
[1 1 1 1 1 1 1 1 1 1 1]	Counter({1: 11})
[2 2 2 2 2 2 2 2 2 2 2]	Counter({2: 11})
[0 0 0 2 2 2 2 2 2 2 2]	Counter({2: 8, 0: 3})
[0 0 0 0 0 0 0 0 0 0 0]	Counter(

In [49]:
for _ in range(len(best_models)):
    torch.save(best_models[_], f'roberta-large{_}.pth')

In [ ]:
torch.load()